In [1]:
import insertsizes as counting
import episcanpy as epi
import pandas as pd

In [2]:
fragments_file = "/mnt/workspace2/jdetlef/data/public_data/fragments_heart_left_ventricle_194_sorted.bed"
h5ad_file = "/mnt/workspace2/jdetlef/data/public_data/heart_lv_SM-JF1NY.h5ad"

In [3]:
adata = epi.read_h5ad(h5ad_file)

In [4]:
adata_barcodes = adata.obs.index.tolist()

In [5]:
%%time
# split index for barcodes CBs
barcodes = []
for entry in adata_barcodes:
    barcode = entry.split('+')[1]
    barcodes.append(barcode)

CPU times: user 2.59 ms, sys: 0 ns, total: 2.59 ms
Wall time: 2.6 ms


In [6]:
%%time
# counter = counting.MPFragmentCounter()
table_mp = counting.insertsize_from_fragments(fragments_file, barcodes, n_threads=10)
print(table_mp)

Count insertsizes from fragments...
Starting counting fragments...


Processing Chunks: 17it [01:08,  4.03s/it]


Done reading file - elapsed time: 0:01:22
Converting counts to dataframe...
Done getting insertsizes from fragments!
                        mean_insertsize  insertsize_count  \
AAATCCGCATAAACGTCCCGTT           200.82              5761   
AAATCCGCATAAATGCTACGGG           162.90             15741   
AAATCCGCATAACATGAAGCGC           191.14              2638   
AAATCCGCATAACCGCTAATGA           174.73              4213   
AAATCCGCATAACTTCGACCAG           184.72             17424   
...                                 ...               ...   
TTCGTCCGACTGCCGTCTCAAC           184.97              3669   
TTCGTCCGACTGCGTCGAGTAC           194.13             11340   
TTCGTCCGACTTGAGTGCTGTG           186.06              3786   
TTCGTCCGACTTTCGAAGAAGG           191.28              5783   
TTCGTCCGACTTTGTGTTACCG           188.35              4693   

                                                                     dist  
AAATCCGCATAAACGTCCCGTT  [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, ... 

In [7]:
import sctoolbox.tools as tools

Unable to determine R home: [Errno 2] No such file or directory: 'R'


In [8]:
%%time
table_sctoolbox = tools._insertsize_from_fragments(fragments_file, barcodes)
print(table_sctoolbox)

[INFO] Counting fragment lengths from fragments file...
[INFO] Done reading file - elapsed time: 0:04:07
[INFO] Converting counts to dataframe...
[INFO] Done getting insertsizes from fragments!
                        insertsize_count  mean_insertsize  0  1  2  3  4  5  \
AGGGATAAACCACCGAAGGTCA              6999           185.35  0  0  0  0  0  0   
AGCGTGTCATTCGCGAGATAGT             18041           183.39  0  4  1  0  0  0   
AAATCCGCATGTCCAGATTTCC             15322           174.45  0  0  0  1  0  0   
TGATTACGCGCCTTTCCGTATC              3066           174.37  0  0  0  0  0  0   
ACCTTCAAGCACTCGTTCCGAT              7780           199.32  0  0  0  0  3  0   
...                                  ...              ... .. .. .. .. .. ..   
TCGGCTTATTTGGACATACTGG              1985           195.80  0  0  0  0  0  0   
CTTCACAGTCCGATATCGATCC              3039           195.34  0  0  0  0  0  0   
GCTCTTGGAAGAATCGGTGTTC              1707           210.02  0  0  0  0  0  0   
ACCCTTATCTTAGATA

In [9]:
def get_dist_df(dist):
    
    table_dict = {}
    for row in dist.iterrows():
        barcode = str(row[0])
        table_dict[barcode] = {}

        for i, counts in enumerate(row[1]['dist']):
            table_dict[barcode][i] = counts
    
    dist_df = pd.DataFrame(table_dict).T
    
    return dist_df

In [10]:
table_sctoolbox = table_sctoolbox[[c for c in table_sctoolbox.columns if isinstance(c, int)]]
table_mp = get_dist_df(table_mp)
sorted_sct_table = table_sctoolbox.sort_index()
sorted_table_mp = table_mp.sort_index()

In [11]:
sorted_sct_table.equals(sorted_table_mp)

True